### 1.	 (40 points) Hand Problem: Solve for the reservoir pressure for a 4 grid-block system and uniform-size grid blocks and time step of 1.0 day. The permeability and porosity in the 4 blocks can be given by:


k =  [75 100 150 125] md 
phi =  [0.15 0.20 0.25 0.22] 

Note: this script below is meant to verify the accuracy and help with the final calculation only. The work by hand is shown on the first page.

In [190]:
import numpy as np
from numpy.linalg import inv

In [191]:
k = np.array([75,100,150,125])
phi = np.array([.15,.20,.25,.22])
A = 80000 #ft^2
Bw = 1
Ct = 1e-5
L = 4000 #ft
Vi = A*L/4 #ft^3
delta_t = 1
delta_x = L/4
mu = 1 #cp 
k_geometric_mean = 2*(1/k[:-1] + 1/k[1:])**-1
half_trans = k_geometric_mean*A/(Bw*mu*delta_x)

B = np.matrix(np.diag(Vi*phi*Ct/Bw))
P0 = np.matrix([[1000]]*4)
Q = np.matrix([[10000],[-10000],[0],[6.33e-3*2*500*10000]])

bc_in = 0 #(0 for Neumann, 1 for Dirichlet)
bc_out = 1
Pout = 500

In [192]:
half_trans

array([  6857.14285714,   9600.        ,  10909.09090909])

In [193]:
k_geometric_mean

array([  85.71428571,  120.        ,  136.36363636])

In [194]:
half_trans_full = half_trans
half_trans_full = np.insert(half_trans_full,0,0)
half_trans_full = np.insert(half_trans_full,len(half_trans_full),20000)
half_trans_full = half_trans_full[:-1] + half_trans_full[1:]
half_trans_full

array([  6857.14285714,  16457.14285714,  20509.09090909,  30909.09090909])

In [195]:
half_trans

array([  6857.14285714,   9600.        ,  10909.09090909])

In [196]:
my_matrix = 0
my_matrix_kplus1 = np.diag(-1*half_trans,k=1)
my_matrix_kminus1 = np.diag(-1*half_trans,k=-1)
my_matrix_k0 = np.diag(half_trans_full,k=0)

my_matrix = my_matrix_kplus1 + my_matrix_kminus1 + my_matrix_k0
T = my_matrix*6.33e-3

In [197]:
T

array([[  43.40571429,  -43.40571429,    0.        ,    0.        ],
       [ -43.40571429,  104.17371429,  -60.768     ,    0.        ],
       [   0.        ,  -60.768     ,  129.82254545,  -69.05454545],
       [   0.        ,    0.        ,  -69.05454545,  195.65454545]])

In [198]:
P0

matrix([[1000],
        [1000],
        [1000],
        [1000]])

In [199]:
Q

matrix([[ 10000.],
        [-10000.],
        [     0.],
        [ 63300.]])

In [200]:
T

array([[  43.40571429,  -43.40571429,    0.        ,    0.        ],
       [ -43.40571429,  104.17371429,  -60.768     ,    0.        ],
       [   0.        ,  -60.768     ,  129.82254545,  -69.05454545],
       [   0.        ,    0.        ,  -69.05454545,  195.65454545]])

In [208]:
B

matrix([[ 120.,    0.,    0.,    0.],
        [   0.,  160.,    0.,    0.],
        [   0.,    0.,  200.,    0.],
        [   0.,    0.,    0.,  176.]])

In [202]:
implicit = inv(T + B/delta_t)*(B*P0/delta_t + Q)
implicit

matrix([[ 1050.61776482],
        [  960.17188029],
        [  955.26193797],
        [  821.36807595]])

In [201]:
explicit = P0 + delta_t*inv(B)*(Q-T*P0)
explicit

matrix([[ 1083.33333333],
        [  937.5       ],
        [ 1000.        ],
        [  640.34090909]])

In [203]:
# Crank Nicholson
theta = .5
crank_nicholson = inv((1-theta)*T + B/delta_t)*((B/delta_t - theta*T )*P0 + Q)
crank_nicholson

matrix([[ 1063.5555162 ],
        [  954.19949727],
        [  964.02642943],
        [  764.29641708]])